# Colaboración de Dos Agentes (Flota Estelar)

## Objetivos de aprendizaje
- Crear dos agentes con roles especializados
- Implementar un flujo de trabajo **secuencial**
- Pasar el output de un agente como input a otro
- Entender cómo dividir tareas complejas entre agentes
- Combinar resultados de múltiples agentes

## Idea clave
**Dos agentes, dos roles especializados, un flujo secuencial:**

```
Tema complejo
    ↓
[Agente Analista] → investigación detallada
    ↓
[Agente Redactor] → resumen ejecutivo
    ↓
Resultado final
```

En la Flota Estelar: un oficial investiga a fondo un tema técnico, y otro oficial de comunicaciones lo resume para que sea comprensible para el Capitán.

## Paso 0: Importaciones y configuración

In [9]:
import os
import time
from agent_framework import ChatAgent, SequentialBuilder, WorkflowOutputEvent
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv

load_dotenv()
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")
print(AZURE_OPENAI_DEPLOYMENT)

print("✅ Entorno cargado y Microsoft Agent Framework importado")

gpt-4
✅ Entorno cargado y Microsoft Agent Framework importado


## Paso 1: Crear dos agentes con roles especializados
- **AnalistaFlota**: Investigador técnico de la Flota
- **RedactorFlota**: Oficial de comunicaciones que traduce lo técnico a comprensible

In [10]:
# Agente 1: Investigador/Analista
analista = ChatAgent(
    chat_client=OpenAIChatClient(
        base_url=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY,
        model_id=AZURE_OPENAI_DEPLOYMENT
    ),
    name="AnalistaFlota",
    instructions="""Eres un analista técnico de la Flota Estelar. Investigas temas complejos en 
    profundidad, proporcionando hallazgos detallados, precisos y basados en protocolos de la Flota."""
)

# Agente 2: Redactor/Comunicador
redactor = ChatAgent(
    chat_client=OpenAIChatClient(
        base_url=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY,
        model_id=AZURE_OPENAI_DEPLOYMENT
    ),
    name="RedactorFlota",
    instructions="""Eres un oficial de comunicaciones de la Flota. Tu tarea es tomar información 
    técnica detallada y crear resúmenes ejecutivos claros, concisos y accesibles para los 
    capitanes y mandos."""

)

print("✅ Dos agentes especializados creados")

✅ Dos agentes especializados creados


## Paso 2: Crear workflow secuencial con SequentialBuilder (API oficial de Microsoft Agent Framework)
Con `SequentialBuilder`, el framework gestiona automáticamente:
- Ejecución secuencial (agente 1 → agente 2)
- Pasaje del contexto completo entre agentes
- Agregación de resultados

In [11]:
# PATRÓN OFICIAL DE MICROSOFT AGENT FRAMEWORK
# SequentialBuilder crea un pipeline donde:
# - Agente 1 recibe: prompt inicial
# - Agente 2 recibe: conversación completa de agente 1 + su output
# - Agente N recibe: toda la conversación anterior

workflow_secuencial = SequentialBuilder().participants([analista, redactor]).build()

print("✅ Workflow secuencial creado con SequentialBuilder")

✅ Workflow secuencial creado con SequentialBuilder


## Paso 3: Ejecutar el workflow secuencial
El workflow ejecuta automáticamente: AnalistaFlota → RedactorFlota

In [12]:
async def flujo_colaborativo():
    tema = "Ventajas de los escudos deflectores de warp en encuentros diplomáticos"
    
    print(f"\n{'='*80}")
    print("FLUJO SECUENCIAL CON SequentialBuilder (Microsoft Agent Framework)")
    print(f"{'='*80}")
    print(f"📊 Tema: {tema}\n")
    print("⏳ Ejecutando workflow secuencial (AnalistaFlota → RedactorFlota)...\n")
    
    inicio = time.time()
    
    # Ejecutar workflow y capturar eventos
    output_evt = None
    async for event in workflow_secuencial.run_stream(tema):
        if isinstance(event, WorkflowOutputEvent):
            output_evt = event
            break
    
    tiempo_total = time.time() - inicio
    
    print(f"✅ Workflow completado en {tiempo_total:.2f}s\n")
    
    if output_evt:
        messages = output_evt.data
        
        print(f"{'='*80}")
        print("CONVERSACIÓN COMPLETA (todas las fases)")
        print(f"{'='*80}\n")
        
        # Mostrar todos los mensajes del workflow
        for i, msg in enumerate(messages, 1):
            author = msg.author_name or ("usuario" if msg.role.value == "user" else "asistente")
            print(f"{i}. [{author}]:")
            print("-" * 80)
            content = msg.text if hasattr(msg, 'text') else str(msg.content)
            print(f"{content}\n")
        
        return {
            "tema": tema,
            "messages": messages,
            "tiempo_total": tiempo_total
        }

# Ejecutar el flujo colaborativo
resultado = await flujo_colaborativo()


FLUJO SECUENCIAL CON SequentialBuilder (Microsoft Agent Framework)
📊 Tema: Ventajas de los escudos deflectores de warp en encuentros diplomáticos

⏳ Ejecutando workflow secuencial (AnalistaFlota → RedactorFlota)...

✅ Workflow completado en 12.04s

CONVERSACIÓN COMPLETA (todas las fases)

1. [usuario]:
--------------------------------------------------------------------------------
Ventajas de los escudos deflectores de warp en encuentros diplomáticos

2. [AnalistaFlota]:
--------------------------------------------------------------------------------
Como analista técnico de la Flota Estelar, es fundamental evaluar el papel y las ventajas de los **escudos deflectores de warp** en escenarios de encuentros diplomáticos. Los escudos deflectores convencionales son una parte estándar de todas las naves, pero los escudos adaptados o reforzados por el campo warp representan una capa adicional de protección estratégica. A continuación se detallan las ventajas clave dentro de un contexto dipl

## Paso 4: Análisis del flujo
Observa cómo SequentialBuilder simplifica la orquestación.

## Análisis del Flujo Colaborativo

### 📊 Estadísticas de Ejecución

- **Tema investigado**: Ventajas de los escudos deflectores de warp en encuentros diplomáticos
- **Mensajes en la conversación**: Múltiples (AnalistaFlota + RedactorFlota)
- **Tiempo total**: Depende del modelo (generalmente 10-30 segundos)

### ✨ Ventajas de SequentialBuilder

| # | Ventaja | Descripción |
|---|---------|-------------|
| 1 | **API oficial** | Parte de Microsoft Agent Framework nativo |
| 2 | **Código limpio** | 4-5 líneas vs 50+ líneas de orquestación manual |
| 3 | **Contexto automático** | Cada agente recibe la conversación completa anterior |
| 4 | **Streaming en tiempo real** | `WorkflowOutputEvent` proporciona eventos durante ejecución |
| 5 | **Escalable** | Agregar más agentes: `.participants([agente1, agente2, agente3])` |
| 6 | **Documentación oficial** | Respaldado por Microsoft Learn |

### 🔧 Patrón Oficial

```python
workflow = SequentialBuilder().participants([agente1, agente2]).build()

async for event in workflow.run_stream(prompt):
    if isinstance(event, WorkflowOutputEvent):
        resultados = event.data
```

**Flujo de ejecución:**
```
Prompt inicial
    ↓
[Agente 1] recibe: prompt
    ↓
[Agente 2] recibe: conversación de agente 1 + output
    ↓
WorkflowOutputEvent: resultado final agregado
```

## Resumen de aprendizaje

**SequentialBuilder (Microsoft Agent Framework nativo):**
1. **Orquestación automática**: El framework gestiona el flujo agente1 → agente2 → ... → agenteN
2. **Contexto completo**: Cada agente recibe toda la conversación anterior
3. **Código limpio**: Pasa de 50+ líneas manual a 4-5 líneas con la API oficial
4. **Streaming de eventos**: `WorkflowOutputEvent` proporciona resultados agregados
5. **Escalabilidad**: Agregar más agentes es trivial: `.participants([a, b, c, d])`

**En la Flota Estelar:**
- Un oficial investiga aspectos técnicos del tema
- Otro oficial lo resume para el Capitán
- El resultado es más claro, conciso y accionable
- Todo gestionado automáticamente por Microsoft Agent Framework

**Comparativa:**
```python
# ANTES (manual):
resultado1 = await analista.run(consulta)
resultado2 = await redactor.run(f"Resume: {resultado1.text}")
# ... 50+ líneas de orquestación manual

# DESPUÉS (SequentialBuilder):
workflow = SequentialBuilder().participants([analista, redactor]).build()
async for event in workflow.run_stream(tema):
    if isinstance(event, WorkflowOutputEvent):
        resultados = event.data  # ¡Hecho!
```

**Próximos patrones:** ConcurrentBuilder (paralelo), GroupChatBuilder (iterativo), MagenticBuilder (planificación)